In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
#import gdal
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Reshape
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
#import tifffile

2023-05-23 16:12:45.223858: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 16:12:45.310055: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-23 16:12:45.310066: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-23 16:12:45.327113: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-23 16:12:45.799300: W tensorflow/stream_executor/platform/de

### Patchng sample code for one image

### Do not run untill there is a need to generate the patch images for train, label and test images.
***************************************************************************************************

In [10]:
import os
from osgeo import gdal, gdal_array

def patch_images(input_dir, output_dir, patch_size, overlap):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get a list of all image files in the input directory
    image_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]

    for image_file in image_files:
        # Construct the input and output file paths
        input_path = os.path.join(input_dir, image_file)

        # Open the input image using GDAL and get its properties
        dataset = gdal.Open(input_path, gdal.GA_ReadOnly)
        if dataset is None:
            print(f"Error opening {input_path}. Skipping...")
            continue

        width = dataset.RasterXSize
        height = dataset.RasterYSize
        bands = dataset.RasterCount

        # Calculate the number of patches in each dimension
        num_patches_x = (width - overlap) // (patch_size - overlap)
        num_patches_y = (height - overlap) // (patch_size - overlap)

        for i in range(num_patches_x):
            for j in range(num_patches_y):
                x_start = i * (patch_size - overlap)
                y_start = j * (patch_size - overlap)
                x_end = x_start + patch_size
                y_end = y_start + patch_size

                patch_data = dataset.ReadAsArray(x_start, y_start, patch_size, patch_size)

                patch_output_path = os.path.join(
                    output_dir, f"{os.path.splitext(image_file)[0]}_{i}_{j}.tif"
                )

                driver = gdal.GetDriverByName("GTiff")
                patch_output = driver.Create(
                    patch_output_path,
                    patch_size,
                    patch_size,
                    bands,
                    gdal_array.NumericTypeCodeToGDALTypeCode(patch_data.dtype)
                )

                # Set the geotransform and projection from the input image
                patch_output.SetGeoTransform(dataset.GetGeoTransform())
                patch_output.SetProjection(dataset.GetProjection())

                for k in range(bands):
                    patch_output.GetRasterBand(k + 1).WriteArray(patch_data[k])

                patch_output.FlushCache()

        dataset = None  # Close the input image dataset

### Loading for the train images

In [3]:
# Set the input and output directories and patch parameters
input_directory = 'data/train/images/'
output_directory = "data/train_patch/"
patch_size = 512
overlap = 256

# Patch the images in the directory
patch_images(input_directory, output_directory, patch_size, overlap)

512

### Loading for the test images

In [11]:
# Set the input and output directories and patch parameters
input_directory = 'data/test/images/'
output_directory = "data/test_patch/"
patch_size = 512
overlap = 256

# Patch the images in the directory
patch_images(input_directory, output_directory, patch_size, overlap)

### Loading for the train labels

In [8]:
import os
from osgeo import gdal, gdal_array

def patch_images_gs(input_dir, output_dir, patch_size, overlap):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get a list of all image files in the input directory
    image_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]

    for image_file in image_files:
        # Construct the input and output file paths
        input_path = os.path.join(input_dir, image_file)

        # Open the input image using GDAL and get its properties
        dataset = gdal.Open(input_path, gdal.GA_ReadOnly)
        if dataset is None:
            print(f"Error opening {input_path}. Skipping...")
            continue

        width = dataset.RasterXSize
        height = dataset.RasterYSize
        bands = dataset.RasterCount

        # Make sure the input image is grayscale (single band)
        if bands != 1:
            print(f"Input image {input_path} is not grayscale. Skipping...")
            continue

        # Calculate the number of patches in each dimension
        num_patches_x = (width - overlap) // (patch_size - overlap)
        num_patches_y = (height - overlap) // (patch_size - overlap)

        for i in range(num_patches_x):
            for j in range(num_patches_y):
                x_start = i * (patch_size - overlap)
                y_start = j * (patch_size - overlap)
                x_end = x_start + patch_size
                y_end = y_start + patch_size

                patch_data = dataset.ReadAsArray(x_start, y_start, patch_size, patch_size)

                patch_output_path = os.path.join(
                    output_dir, f"{os.path.splitext(image_file)[0]}_{i}_{j}.tif"
                )

                driver = gdal.GetDriverByName("GTiff")
                patch_output = driver.Create(
                    patch_output_path,
                    patch_size,
                    patch_size,
                    bands,
                    gdal_array.NumericTypeCodeToGDALTypeCode(patch_data.dtype)
                )

                # Set the geotransform and projection from the input image
                patch_output.SetGeoTransform(dataset.GetGeoTransform())
                patch_output.SetProjection(dataset.GetProjection())

                patch_output.GetRasterBand(1).WriteArray(patch_data)

                patch_output.FlushCache()

        dataset = None  # Close the input image dataset

In [9]:
# Set the input and output directories and patch parameters
input_directory = 'data/train/gt/'
output_directory = "data/train_label_patch/"
patch_size = 512
overlap = 256

# Patch the images in the directory
patch_images_gs(input_directory, output_directory, patch_size, overlap)

***************************************************************************************************

### Code for the full data load - Do not delete

### Load the data from the patch folders

In [6]:
# Function to load image data using GDAL
def load_image(image_path):
    dataset = gdal.Open(image_path, gdal.GA_ReadOnly)
    image_data = dataset.ReadAsArray()
    dataset = None
    return image_data

# Function to load image dataset
def load_dataset(data_dir):
    # Get list of image file paths
    image_files = [f for f in os.listdir(data_dir) if f.endswith('.tif')]

    # Load images and concatenate into a single numpy array
    images = []
    for image_file in image_files:
        image_path = os.path.join(data_dir, image_file)
        image_data = load_image(image_path)
        images.append(image_data)
    images = np.stack(images, axis=0)

    return images

# Function to load a subset of the dataset
def load_subset(data_dir, start_index, end_index):
    image_files = [f for f in os.listdir(data_dir) if f.endswith('.tif')]
    images = []
    for i in range(start_index, end_index):
        image_file = image_files[i]
        image_path = os.path.join(data_dir, image_file)
        image_data = load_image(image_path)
        images.append(image_data)
    images = np.stack(images, axis=0)
    return images

### Load raw images using batch wise

### Sliting the data if raw images or raw batch of images to be trained

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def train_generator(x_train_dir, y_train_dir, target_size=(128, 128), batch_size=32):
    """A generator that returns image and mask arrays.
    
    Args:
    x_train_dir (str): Directory of the training images.
    y_train_dir (str): Directory of the ground truth masks.
    target_size (tuple): Size of the images/masks that the generator should return.
    batch_size (int): Number of samples per batch.
    """
    
    # List of file names
    x_filenames = os.listdir(x_train_dir)
    y_filenames = os.listdir(y_train_dir)

    # Total number of samples
    total_samples = len(x_filenames)
    
    # Ensure that we have the same number of images and masks
    assert len(x_filenames) == len(y_filenames), \
    'Number of training images and masks should be the same.'
    
    while True:
        # Shuffle indices to minimize overfitting
        indices = np.random.permutation(total_samples)
        
        # Create batches
        for i in range(0, total_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            
            batch_x = []
            batch_y = []
            
            for j in batch_indices:
                # Load image and mask
                x_path = os.path.join(x_train_dir, x_filenames[j])
                y_path = os.path.join(y_train_dir, y_filenames[j])
                
                x = load_img(x_path, target_size=target_size)  # this is PIL image
                y = load_img(y_path, target_size=target_size, color_mode='grayscale')  # grayscale for masks
                
                # Convert PIL image to numpy array
                x = img_to_array(x)
                y = img_to_array(y)
                
                # Normalize
                x /= 255.
                y /= 255.
                
                batch_x.append(x)
                batch_y.append(y)
            
            batch_x = np.array(batch_x)
            batch_y = np.array(batch_y)
            
            yield (batch_x, batch_y)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
x_train_dir = 'data/train_patch/'
y_train_dir = 'data/train_label_patch'

# Image size you want to generate
img_width = 256
img_height = 256

# Instantiate the generator
generator = train_generator(x_train_dir, y_train_dir)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras import layers, models

def unet_model():
    model = Sequential()

    # Encoder
    #model.add(layers.Reshape((5000*5000*3, ), input_shape=(5000,5000, 3)))
    #model.add(Reshape((5000*5000*3,), input_shape=(5000,5000,3))) 

    model.add(Conv2D(64, 3, activation='relu', padding='same', input_shape = ((5000, 5000, 3))))
    model.add(Conv2D(64, 3, activation='relu', padding='same'))
    #model.add(Conv2D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    #model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    #model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Decoder
    model.add(UpSampling2D(size=(2, 2)))
    #model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(Conv2D(128, 3, activation='relu', padding='same'))

    model.add(UpSampling2D(size=(2, 2)))
    #model.add(Conv2D(64, 3, activation='relu', padding='same'))
    model.add(Conv2D(64, 3, activation='relu', padding='same'))

    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, 3, activation='sigmoid', padding='same'))

    return model

In [7]:
# Create the U-Net model
model = unet_model()

2023-05-23 16:13:51.904480: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-23 16:13:51.904593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-23 16:13:51.904638: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-23 16:13:51.904680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-23 16:13:51.904721: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Use the generator to train the model
history = model.fit(generator, epochs=1, steps_per_epoch=5, verbose = 1, batch_size = 8)

2023-05-23 16:13:55.119937: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51200000000 exceeds 10% of free system memory.


In [1]:
#train_images_reshaped = train_images.reshape((len(train_images), 5000, 5000, 3))
#train_labels_reshaped = train_labels.reshape((len(train_labels), 5000, 5000, 3))

In [17]:
#train_images = train_images.reshape((16,5000, 5000, 3))

In [18]:
#train_labels =train_labels.reshape((16,5000, 5000, 1))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(val_images, val_labels)
print('Validation Loss:', loss)
print('Validation Accuracy:', accuracy)

# Plot the training and validation loss curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Save the trained model
model.save("roof_suitability_model")